In [1]:
import feather
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

from sklearn.preprocessing import MinMaxScaler
from datetime import date
from currency_converter import CurrencyConverter

# bokeh
# from bokeh.resources import INLINE
# from bokeh.plotting import figure, gridplot, GridSpec, output_file, show
# from bokeh.io import output_notebook
# output_notebook(resources=INLINE)

# dask
# import dask
# import dask.dataframe as dd
# from dask.delayed import delayed

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline

In [2]:
def convert_to_mxn(montos, monedas, fechas):
    monto_pesos = []
    for monto, moneda, fecha in zip(montos, monedas, fechas):
        if moneda in {'MXN', 'TEST', 'OTH'}:
            monto_pesos.append(monto)
        else:
            val = converter.convert(monto, moneda, 'MXN', date=fecha)
            monto_pesos.append(val)
    return monto_pesos

# lectura y tirar repetidos
df_procedimientos = feather.read_dataframe('../data/processed/procedimientos_2017_06_14.feather')


print(df_procedimientos.shape)
df_procedimientos = df_procedimientos.loc[df_procedimientos.GOBIERNO == 'APF', :]
df_procedimientos = df_procedimientos.drop('GOBIERNO', axis=1)
print(df_procedimientos.shape)

dep_repetidas = {
    'HOSPITAL GENERAL DE MEXICO "DR. EDUARDO LICEAGA"': 'HOSPITAL GENERAL DE MEXICO',
    'INSTITUTO NACIONAL DE REHABILITACION LUIS GUILLERMO IBARRA IBARRA': 'INSTITUTO NACIONAL DE REHABILITACION',
    'CENTRO DE INVESTIGACION EN GEOGRAFIA Y GEOMATICA, "ING. JORGE L. TAMAYO", A.C.': 'CENTRO DE INVESTIGACION EN GEOGRAFIA Y GEOMATICA "ING. JORGE L. TAMAYO", A.C.',
    'TRIBUNAL FEDERAL DE JUSTICIA FISCAL Y ADMINISTRATIVA': 'TRIBUNAL FEDERAL DE JUSTICIA ADMINISTRATIVA',
    'INSTITUTO DE INVESTIGACIONES ELECTRICAS': 'INSTITUTO NACIONAL DE ELECTRICIDAD Y ENERGIAS LIMPIAS'
}



for k, v in dep_repetidas.items():
    df_procedimientos.loc[df_procedimientos.DEPENDENCIA == k, 'DEPENDENCIA'] = v

del dep_repetidas

cols_id = ['NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO', 'CODIGO_EXPEDIENTE']
df_validos = df_procedimientos.drop_duplicates(subset=cols_id, keep='first')
print(df_validos.shape)


# convertir a Pesos
converter = CurrencyConverter(fallback_on_missing_rate=True)
monto_en_pesos = convert_to_mxn(
    df_validos.IMPORTE_CONTRATO.values, df_validos.MONEDA.values, df_validos.FECHA_INICIO)


df_validos = df_validos.assign(IMPORTE_PESOS=monto_en_pesos)
df_validos = df_validos.drop('IMPORTE_CONTRATO', axis=1)

del monto_en_pesos

# Limpiar CLAVEUC
df_validos.loc[df_validos.CLAVEUC.str.len() == 8, 'CLAVEUC'] = df_validos.CLAVEUC.str.pad(9, fillchar='0')
df_validos.loc[df_validos.CLAVEUC.str.len() == 7, 'CLAVEUC'] = df_validos.CLAVEUC.str.pad(9, fillchar='0')

df_validos.head()

(861169, 44)
(742022, 43)


## Ver CONVENIO_MODIFICATORIOs 

In [3]:
df_procedimientos.CONVENIO_MODIFICATORIO.value_counts(dropna=False)

 0.0    607056
NaN     120284
 1.0     14682
Name: CONVENIO_MODIFICATORIO, dtype: int64

## se puede hacer la seleccions de los cols_id + convenios y sacar el nunique para ver cuales tienen más de un tipo de convenio

## Hacer análisis temporal (días en los que se apruebas contratos)

In [5]:
## RFC nulos y con monto
df_registro = pd.read_excel(
    '../data/raw/Registro Único de Proveedores y Contratistas.xlsx',
    dtype={'FOLIO_RUPC': str}
)
df_registro.loc[:, 'FECHA_INSCRIPCION_RUPC'] = pd.to_datetime(df_registro.FECHA_INSCRIPCION_RUPC, yearfirst=False)
df_registro.loc[
    :, 'NOMBRE_EMPRESA'] = df_registro.NOMBRE_EMPRESA.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.upper()
print(df_registro.shape)
df_registro = df_registro.loc[df_registro.RFC.isnull()]
print(df_registro.shape)
df_registro.head()

(8649, 16)
(2454, 16)


,FOLIO_RUPC,RFC,NOMBRE_EMPRESA,PAIS,ENTIDAD_FEDERATIVA,ESTRATIFICACION,TIPO_USUARIO,SECTOR,GIRO,CONTRATOS,FECHA_INSCRIPCION_RUPC,CONTRATOS_EVALUADOS_ADQ,GRADO_CUMPLIMIENTO_ADQ,CONTRATOS_EVALUADOS_OP,GRADO_CUMPLIMIENTO_OP,SITIO_WEB
322,51081,NaN,IVAN RODOLFO BALANDRANO HERNANDEZ,MX,MX-DF,No MIPYME,Proveedor,Comercio,COMERCIO AL POR MAYOR DE ARTICULOS DE PAPELERI...,150,2016-04-05,NaN,NaN,NaN,NaN,NaN
340,44277,NaN,ALAN RIOS CRUZ,MX,MX-DF,Pequeña,NaN,NaN,PAPEL Y CARTON,146,2014-12-18,NaN,NaN,NaN,NaN,NaN
365,9415,NaN,ROBERTO MARTINEZ CASTRO,MX,MX-DF,Micro,Proveedor,Comercio,"COMERCIALIZADORA DE MATERIAL DE LIMPIEZA, PAPE...",137,2016-04-05,2.0,100.0,NaN,NaN,NaN
456,37945,NaN,BRYAN RAMIREZ SOTO,MX,MX-DF,Pequeña,Proveedor,Servicio,"INDUSTRIAS CONEXAS A LA IMPRESION, COMO LA ENC...",112,2014-12-18,NaN,NaN,NaN,NaN,NaN
458,13484,NaN,SERGIO OCTAVIO PAEZ LIZARRAGA,MX,MX-BCS,Micro,Proveedor,Industria,VENTA DE REFACCIONES Y REPARACION DE EQUIPOS H...,112,2014-12-09,NaN,NaN,NaN,NaN,NaN


In [20]:
df_registro.FOLIO_RUPC.nunique()

2454

In [7]:
df_rfc_nulos = df_validos.loc[df_validos.FOLIO_RUPC.isin(df_registro.FOLIO_RUPC.unique())]

In [22]:
df_rfc_nulos.FOLIO_RUPC.nunique(), df_rfc_nulos.DEPENDENCIA.nunique()

(1887, 203)

In [8]:
df_registro[df_registro.NOMBRE_EMPRESA == 'JESUS JOSE HERNANDEZ PINON']

,FOLIO_RUPC,RFC,NOMBRE_EMPRESA,PAIS,ENTIDAD_FEDERATIVA,ESTRATIFICACION,TIPO_USUARIO,SECTOR,GIRO,CONTRATOS,FECHA_INSCRIPCION_RUPC,CONTRATOS_EVALUADOS_ADQ,GRADO_CUMPLIMIENTO_ADQ,CONTRATOS_EVALUADOS_OP,GRADO_CUMPLIMIENTO_OP,SITIO_WEB
1786,6146,NaN,JESUS JOSE HERNANDEZ PINON,MX,MX-CHI,Mediana,Contratista,Otro,"CONTRUCCION DE TODO TIPO DE OBRAS, MOVIMIENTOS...",27,2015-05-13,NaN,NaN,1.0,100.0,NINGUNA


### JESUS JOSE HERNANDEZ PINON trabajó en CONAGUA del 97 a 2001, las empresas que menciona en linkedin no las encuentro
### Creo que no hay pedo con élANGEL SANTIAGO SOSA	 también se ve raro, vive en una cerrada en el DF, y no se ve tan chido. RFC en internet SASA-710802-ED0

In [10]:
## Hacer el groupby PROVEEDOR_CONTRATISTA y DEPEndencia
df_rfc_nulos.groupby(['PROVEEDOR_CONTRATISTA', 'DEPENDENCIA']).IMPORTE_PESOS.sum().sort_values(ascending=False).reset_index()

,PROVEEDOR_CONTRATISTA,DEPENDENCIA,IMPORTE_PESOS
0,ALEJANDRO GARZA ESPINOSA,COMISION FEDERAL DE ELECTRICIDAD,1.185366e+09
1,ANGEL CESAR CARO MONTES,COMISION FEDERAL DE ELECTRICIDAD,3.717474e+08
2,ALEJANDRO MORALES SANCHEZ,COMISION FEDERAL DE ELECTRICIDAD,3.050842e+08
3,ARNULFO ALEJANDRO MENCHACA,COMISION FEDERAL DE ELECTRICIDAD,2.053272e+08
4,ALFONSO NAVA BURGOS,"LICONSA, S.A. DE C.V.",2.045082e+08
5,MARIA CECILIA PORTILLO VALDEPENA,"LICONSA, S.A. DE C.V.",1.643135e+08
6,JOSE LUIS FLORES VELAZQUEZ,SECRETARIA DE COMUNICACIONES Y TRANSPORTES,1.058844e+08
7,ISAC GARCIA RODRIGUEZ,SECRETARIA DE COMUNICACIONES Y TRANSPORTES,1.058152e+08
8,JUAN MANUEL SALAS ALMODOVAR,SECRETARIA DE COMUNICACIONES Y TRANSPORTES,9.080108e+07
9,ABRAHAM RODRIGUEZ HERRERA,SECRETARIA DE COMUNICACIONES Y TRANSPORTES,8.485604e+07


In [33]:
df_rfc_nulos.groupby('PROVEEDOR_CONTRATISTA').IMPORTE_PESOS.sum().sort_values(ascending=False)

PROVEEDOR_CONTRATISTA
ALEJANDRO GARZA ESPINOSA                  1.185366e+09
ANGEL CESAR CARO MONTES                   3.717474e+08
ALEJANDRO MORALES SANCHEZ                 3.050842e+08
ARNULFO ALEJANDRO MENCHACA                2.053272e+08
ALFONSO NAVA BURGOS                       2.045082e+08
MARIA CECILIA PORTILLO VALDEPENA          1.643135e+08
JOSE LUIS FLORES VELAZQUEZ                1.058844e+08
ISAC GARCIA RODRIGUEZ                     1.058152e+08
JUAN MANUEL SALAS ALMODOVAR               9.080108e+07
ABRAHAM RODRIGUEZ HERRERA                 8.485604e+07
DONATO SANCHEZ VELEZ                      7.760538e+07
JESUS JOSE HERNANDEZ PINON                7.182785e+07
RUBEN TINOCO VAZQUEZ                      6.631125e+07
ELSA GUADALUPE CASTELAZO Y SERRADEL       6.578150e+07
MIGUEL CHAN GARCIA                        5.293247e+07
JORGE JAVIER ORTIZ CABRERA                5.165967e+07
CONSTANTINO RAMON VALCARCEL               5.137031e+07
GUSTAVO DE LA ROSA DEL VALLE              5

In [32]:
df_rfc_nulos.groupby('PROVEEDOR_CONTRATISTA').NUMERO_PROCEDIMIENTO.count().sort_values(ascending=False)

PROVEEDOR_CONTRATISTA
ALAN RIOS CRUZ                            121
SERGIO OCTAVIO PAEZ LIZARRAGA             110
ROBERTO MARTINEZ CASTRO                    99
BRYAN RAMIREZ SOTO                         85
IVAN RODOLFO BALANDRANO HERNANDEZ          83
MARCO ANTONIO CUBILLAS PUNARO              82
CECILIA FERNANDEZ GUERRERO                 79
LORENZO BAZAN SANTIAGO                     72
MARIO ENRIQUE HERNANDEZ GARCIA             71
MARA PATRICIA SANDOVAL GALINDO             65
ALEJANDRO GARZA ESPINOSA                   65
GEORGINA DE LOURDES CORTES GRAJALES        64
JESUS ARMANDO RAMIREZ ROMERO               64
JORGE ALBERTO RODRIGUEZ ZAZUETA            64
LUIS JESUS OVIEDO GARCIA                   63
RAFAEL HUMBERTO GUILLERMO PEDRERO          61
ARTURO LOPEZ MIRANDA                       60
FLORENCIA PATRICIA TORRES GODINEZ          58
MARCO POLO APIS DIAZ                       57
JUAN MANUEL FERNANDEZ DE LARA CASTILLO     55
MARIA DEL PILAR TREJO MADRIGAL             55
EDGAR ANTONI

In [25]:
# df_rfc_nulos.drop_duplicates(subset=['PROVEEDOR_CONTRATISTA', 'FOLIO_RUPC']).groupby('PROVEEDOR_CONTRATISTA').FOLIO_RUPC.count().sort_values(ascending=False)

In [10]:
# df_registro.loc[df_registro.NOMBRE_EMPRESA == 'JUAN RAMON GARCIA SANCHEZ']

,FOLIO_RUPC,RFC,NOMBRE_EMPRESA,PAIS,ENTIDAD_FEDERATIVA,ESTRATIFICACION,TIPO_USUARIO,SECTOR,GIRO,CONTRATOS,FECHA_INSCRIPCION_RUPC,CONTRATOS_EVALUADOS_ADQ,GRADO_CUMPLIMIENTO_ADQ,CONTRATOS_EVALUADOS_OP,GRADO_CUMPLIMIENTO_OP,SITIO_WEB
5422,435923,NaN,JUAN RAMON GARCIA SANCHEZ,MX,MX-MEX,Micro,Ambos,Construcción,"Diseño, construccion y supervisión de todo tip...",4,2016-04-05,1.0,100.0,NaN,NaN,NaN
7071,426701,NaN,JUAN RAMON GARCIA SANCHEZ,MX,MX-TMS,Micro,Proveedor,Servicio,asesorías,2,2016-04-05,NaN,NaN,NaN,NaN,NaN


In [5]:
df_registro.shape, df_registro.FOLIO_RUPC.nunique(), df_registro.NOMBRE_EMPRESA.nunique()

((8649, 16), 8649, 8648)

In [6]:
df_registro.TIPO_USUARIO.value_counts(dropna=False)

Proveedor      4950
Contratista    2507
Ambos           969
NaN             223
Name: TIPO_USUARIO, dtype: int64

In [6]:
df_registro.FOLIO_RUPC.nunique()

2454

## Número de unidades y errores en el reporte de claveuc


In [12]:
# df_validos.loc[df_validos.CLAVEUC.str.len() != 9].SIGLAS.value_counts()

In [5]:
# UC con más de una dependencia
# en_comun = df_validos.loc[
#     :, ['CLAVEUC', 'DEPENDENCIA', 'SIGLAS']
# ].drop_duplicates(
#     subset=['CLAVEUC', 'DEPENDENCIA']
# ).groupby('CLAVEUC').DEPENDENCIA.count().sort_values(ascending=False).head(1).index

# df_test = df_validos.loc[
#     :, ['CLAVEUC', 'DEPENDENCIA']].drop_duplicates(subset=['CLAVEUC', 'DEPENDENCIA'])
# df_test.loc[df_test.CLAVEUC.isin(en_comun)].sort_values('CLAVEUC').DEPENDENCIA.unique()

In [3]:
df_unidades = df_validos.groupby('DEPENDENCIA', as_index=False).agg({'CLAVEUC': 'nunique'})
df_unidades = df_unidades.sort_values('CLAVEUC', ascending=False)
df_unidades = df_unidades.rename(columns={'CLAVEUC': 'TOTAL_UC'})
df_unidades.to_csv(
    '../data/results/unidades_por_dependencia.psv', index=False, encoding='utf-8', sep='|', quoting=1)
# df_unidades = pd.read_csv('../data/results/unidades_por_dependencia.psv', sep='|')
df_unidades.head(5)

,DEPENDENCIA,TOTAL_UC
68,COMISION FEDERAL DE ELECTRICIDAD,368
159,INSTITUTO MEXICANO DEL SEGURO SOCIAL,115
149,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,91
223,SECRETARIA DE COMUNICACIONES Y TRANSPORTES,76
83,COMISION NACIONAL DEL AGUA,72


In [11]:
pd.Series(df_validos.CLAVEUC.unique()).str.len().value_counts().sum()

1780

264

In [8]:
# with plt.style.context('seaborn-talk'):
#     df_unidades.head(30).set_index('DEPENDENCIA').sort_values('TOTAL_UC').plot.barh(figsize=(14, 9))

In [18]:
df_dir_fed = pd.read_excel('../data/raw/Directorio Unidades Compradoras Gobierno Federal.xls', dtype=str)
df_dir_fed.loc[
    :, 'Dependencia o Entidad'
] = df_dir_fed['Dependencia o Entidad'].str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.upper()

print(df_dir_fed.shape)
df_dir_fed.head()

(1930, 12)


,Nombre UC en CompraNet,Clave UC en CompraNet,Pertenece al,Dependencia o Entidad,Siglas,Página Web,Dirección,CP,Entidad Federativa,Delegación o municipio,Telefono de la UC,Nombre del Responsable
0,AFSEDF-ADMINISTRACIÓN FEDERAL DE SERVICIOS EDU...,011C00999,APF,ADMINISTRACION FEDERAL DE SERVICIOS EDUCATIVOS...,AFSEDF,www.sep.gob.mx,Río Rhin 9 Piso 8,06500,Ciudad de México,Cuauhtémoc,36017100 ext. 40302,Ricardo Rafael Nieto Hernández
1,API-Altamira-Departamento de Concursos y Contr...,009J2Y001,APF,"ADMINISTRACION PORTUARIA INTEGRAL DE ALTAMIRA,...",API-Altamira,www.puertoaltamira.com.mx,"Calle Rios Tamesi Km 0 800, lado sur, Colonia ...",89603,Tamaulipas,Altamira,18332606060,Alberto Trejo Saldaña
2,API-Altamira-DEPARTAMENTO DE RECURSOS MATERIAL...,009J2Y002,APF,"ADMINISTRACION PORTUARIA INTEGRAL DE ALTAMIRA,...",API-Altamira,www.puertoaltamira.com.mx,RIO TAMESI KM 0 800 LADO SUR COLONIA PUERTO IN...,89603,Tamaulipas,Altamira,8332606060,Denisse Ramirez Diaz
3,API-Coatzacoalcos-Coatzacoalcos Jefatura de Pr...,009J3F001,APF,ADMINISTRACION PORTUARIA INTEGRAL DE COATZACOA...,API-Coatzacoalcos,http://www.puertocoatzacoalcos.com.mx/,Interior del Recinto Fiscal s/n antes zona franca,96400,Veracruz,Coatzacoalcos,019212110270,José Manuel Kin Galindo
4,API-Coatzacoalcos-Administración Portuaria Int...,009J3F002,APF,ADMINISTRACION PORTUARIA INTEGRAL DE COATZACOA...,API-Coatzacoalcos,www.puertocoatzacoalcos.com.mx,Interior del Recinto Fiscal Zona Franca S/N,96400,Veracruz,Coatzacoalcos,9212110270,Bernardino Guzmán Zavala


In [19]:
df_dir_fed['Dependencia o Entidad'].nunique(), df_dir_fed['Siglas'].nunique()

(268, 268)

In [27]:
set(df_dir_fed['Dependencia o Entidad'].unique()) - set(df_validos.DEPENDENCIA.unique()) 

{'AGENCIA NACIONAL DE SEGURIDAD INDUSTRIAL Y DE PROTECCION AL MEDIO AMBIENTE DEL SECTOR HIDROCARBUROS ',
 'COMISION NACIONAL DE BIOETICA',
 'FONDO DE EMPRESAS EXPROPIADAS DEL SECTOR AZUCARERO',
 'HOSPITAL GENERAL DE MEXICO "DR. EDUARDO LICEAGA"',
 'INSTITUTO NACIONAL DE REHABILITACION LUIS GUILLERMO IBARRA IBARRA',
 'UNIVERSIDAD AUTONOMA AGRARIA ANTONIO NARRO',
 'UNIVERSIDAD AUTONOMA CHAPINGO'}

In [26]:
df_dir_fed[df_dir_fed['Dependencia o Entidad'].str.startswith('INSTITUTO NACIONAL DE REHABILITACION')]

,Nombre UC en CompraNet,Clave UC en CompraNet,Pertenece al,Dependencia o Entidad,Siglas,Página Web,Dirección,CP,Entidad Federativa,Delegación o municipio,Telefono de la UC,Nombre del Responsable
1211,INR-Subdirección de Conservación y Mantenimien...,012NDF001,APF,INSTITUTO NACIONAL DE REHABILITACION LUIS GUIL...,INR,www.inr.gob.mx,Calzada México Xochimilco No. 289,14389,Ciudad de México,Tlalpan,59991000 ext.18158,Francisco Martínez Cano
1212,INR-Subdirección de Compras y Suministros #012...,012NDF999,APF,INSTITUTO NACIONAL DE REHABILITACION LUIS GUIL...,INR,www.inr.gob.mx,Calzada México-Xochimilco 289,14389,Ciudad de México,Tlalpan,01 55 59991000 Ext.10024,José Raúl Sánchez Arredondo


In [ ]:
# Faltantes
# COMISION NACIONAL DE BIOE
# FONDO DE EMPRESAS EXPROPIADAS DEL SECTOR AZUCARERO
# UNIVERSIDAD AUTONOMA AGRARIA ANTONIO NARRO
# UNIVERSIDAD AUTONOMA CHAPINGO 

In [35]:
df_validos[df_validos['DEPENDENCIA'].str.startswith('UNIVERSIDAD')]


,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,IMPORTE_PESOS
108155,UNIVERSIDAD PEDAGOGICA NACIONAL,UPN,UPN-SECRETARIA ADMINISTRATIVA #011A00001,011A00001,ROMEL CERVANTES ANGELES,MICRO,nan,DIAGONAL ARQUITECTURA,HABILITADO,PEQUENA,NaN,MXN,IA-011A00001-N6-2012,Electrónica,INVITACION A CUANDO MENOS 3 PERSONAS,131414,DESAZOLVE DE 9 FOSAS SEPTICAS Y 7 GRIETAS DE I...,None,SERVICIOS,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,2012-05-22 14:07:00,2012-05-28 10:00:00,2012-05-31,2012-06-01,2012-05-31,2012-12-31,187407,CONTRATACION DEL SERVICIO DE DESASOLVE DE 9 FO...,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,428876.00
108156,UNIVERSIDAD PEDAGOGICA NACIONAL,UPN,UPN-SECRETARIA ADMINISTRATIVA #011A00001,011A00001,ROMEL CERVANTES ANGELES,MICRO,nan,TRANSMANTEIN,HABILITADO,MEDIANA,NaN,MXN,LA-011A00001-I3-2012,Electrónica,LICITACION PUBLICA,110984,LOTE 1. CONSUMIBLES INFORMATICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Internacional,0.0,0.0,2012-03-13 23:30:00,2012-03-23 11:00:00,2012-03-28,2012-03-29,2012-04-01,2012-12-31,155905,"ADQUISICION DE REFACCIONES, CONSUMIBLES INFORM...",Z15122015 03. Licitación Pública Internacional...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,0.0,218361.00
108157,UNIVERSIDAD PEDAGOGICA NACIONAL,UPN,UPN-SECRETARIA ADMINISTRATIVA #011A00001,011A00001,ROMEL CERVANTES ANGELES,MICRO,nan,SANSIBA SA DE CV,HABILITADO,MICRO,NaN,MXN,LA-011A00001-I3-2012,Electrónica,LICITACION PUBLICA,110983,LOTE 2. MATERIAL DE REPROGRAFIA,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Internacional,0.0,0.0,2012-03-13 23:30:00,2012-03-23 11:00:00,2012-03-28,2012-03-30,2012-04-01,2012-12-31,155905,"ADQUISICION DE REFACCIONES, CONSUMIBLES INFORM...",Z15122015 03. Licitación Pública Internacional...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,0.0,61626.90
108158,UNIVERSIDAD PEDAGOGICA NACIONAL,UPN,UPN-SECRETARIA ADMINISTRATIVA #011A00001,011A00001,ROMEL CERVANTES ANGELES,MICRO,nan,DISTRIBUIDORA COMPUTACIONAL JERB SA DE CV,HABILITADO,MICRO,NaN,MXN,LA-011A00001-I3-2012,Electrónica,LICITACION PUBLICA,110982,LOTE 3. REFACCIOES Y ACCESORIOS PARA EQUIPO DE...,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Internacional,0.0,0.0,2012-03-13 23:30:00,2012-03-23 11:00:00,2012-03-28,2012-03-30,2012-04-01,2012-12-31,155905,"ADQUISICION DE REFACCIONES, CONSUMIBLES INFORM...",Z15122015 03. Licitación Pública Internacional...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,0.0,600067.47
108159,UNIVERSIDAD PEDAGOGICA NACIONAL,UPN,UPN-SECRETARIA ADMINISTRATIVA #011A00001,011A00001,ROMEL CERVANTES ANGELES,MICRO,nan,JLJ SA DE CV,HABILITADO,PEQUENA,NaN,MXN,AA-011A00001-N16-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,182047,MATERIAL Y UTILES DE OFICINA,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-09-11 00:00:00,NaT,2012-03-01,2012-03-01,2012-09-11,254777,MATERIAL Y UTILES DE OFICINA,Z15122015 07. Adjudicación Directa Nacional Ar...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,101389.00
108160,UNIVERSIDAD PEDAGOGICA NACIONAL,UPN,UPN-SECRETARIA ADMINISTRATIVA #011A00001,011A00001,ROMEL CERVANTES ANGELES,MICRO,nan,JOSE GARCIA GONZALEZ,HABILITADO,PEQUENA,NaN,MXN,IA-011A00001-N5-2012,Mixta,INVITACION A CUANDO MENOS 3 PERSONAS,131427,SERVIO DE MANTO. PREVENTIVO Y CORRECTIVO AL PA...,None,SERVICIOS,EX

In [3]:
conteo = df_validos.groupby('PROVEEDOR_CONTRATISTA', as_index=False).NUMERO_PROCEDIMIENTO.count()
monto = df_validos.groupby('PROVEEDOR_CONTRATISTA', as_index=False).IMPORTE_PESOS.sum()

df_empresas = pd.merge(conteo, monto, how='inner', on='PROVEEDOR_CONTRATISTA')
df_empresas = df_empresas.sort_values('IMPORTE_PESOS', ascending=False)
df_empresas = df_empresas.rename(
    columns={'NUMERO_PROCEDIMIENTO': 'PROCEDIMIENTOS', 'IMPORTE_PESOS': 'MONTO_PESOS'})
df_empresas = df_empresas.assign(MONTO_por_PROC=df_empresas.MONTO_PESOS.divide(df_empresas.PROCEDIMIENTOS))
del conteo, monto
print(df_empresas.shape)
df_empresas.head()

(140983, 4)


,PROVEEDOR_CONTRATISTA,PROCEDIMIENTOS,MONTO_PESOS,MONTO_por_PROC
58620,GRUPO FARMACOS ESPECIALIZADOS SA DE CV,1953,6.552528e+10,3.355109e+07
48331,"FARMACEUTICOS MAYPO, S.A. DE C.V.",2038,2.730809e+10,1.339945e+07
38176,DISTRIBUIDORA INTERNACIONAL DE MEDICAMENTOS Y ...,1389,2.086963e+10,1.502493e+07
128929,SIXSIGMA NETWORKS MEXICO,64,1.988804e+10,3.107506e+08
124269,SEGUROS INBURSA SA GRUPO FINANCIERO INBUR SA,244,1.788541e+10,7.330087e+07


In [4]:
monto = df_validos.groupby('DEPENDENCIA', as_index=False).IMPORTE_PESOS.sum()
conteo = df_validos.groupby('DEPENDENCIA', as_index=False).NUMERO_PROCEDIMIENTO.count()
df_dependencias = pd.merge(conteo, monto, on='DEPENDENCIA', how='inner')
df_dependencias = df_dependencias.sort_values('IMPORTE_PESOS', ascending=False)
df_dependencias = df_dependencias.set_index('DEPENDENCIA')
df_dependencias = df_dependencias.rename(
    columns={'NUMERO_PROCEDIMIENTO': 'PROCEDIMIENTOS', 'IMPORTE_PESOS': 'MONTO_PESOS'})
df_dependencias = df_dependencias.assign(
    MONTO_por_PROC=df_dependencias.MONTO_PESOS.divide(df_dependencias.PROCEDIMIENTOS))

print(df_dependencias.shape)
del monto, conteo
df_dependencias.head()

(269, 3)


,PROCEDIMIENTOS,MONTO_PESOS,MONTO_por_PROC
DEPENDENCIA,,,
COMISION FEDERAL DE ELECTRICIDAD,141872,3.920760e+11,2.763589e+06
INSTITUTO MEXICANO DEL SEGURO SOCIAL,104022,3.265710e+11,3.139442e+06
SECRETARIA DE COMUNICACIONES Y TRANSPORTES,26364,3.010606e+11,1.141938e+07
INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE LOS TRABAJADORES DEL ESTADO,37894,1.274725e+11,3.363924e+06
PETROLEOS MEXICANOS (CORPORATIVO),17013,7.064321e+10,4.152308e+06


In [5]:
top_5_dep = set(df_dependencias.sort_values('MONTO_PESOS', ascending=False).index[:5].values)
top_5_dep

{'COMISION FEDERAL DE ELECTRICIDAD',
 'INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE LOS TRABAJADORES DEL ESTADO',
 'INSTITUTO MEXICANO DEL SEGURO SOCIAL',
 'PETROLEOS MEXICANOS (CORPORATIVO)',
 'SECRETARIA DE COMUNICACIONES Y TRANSPORTES'}

## Network CFE

In [18]:
dep = 'COMISION FEDERAL DE ELECTRICIDAD'
df_network = df_validos.loc[
    df_validos.DEPENDENCIA == dep, ['CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'IMPORTE_PESOS']
].groupby(
    ['CLAVEUC', 'PROVEEDOR_CONTRATISTA']
).IMPORTE_PESOS.sum().reset_index().sort_values('IMPORTE_PESOS', ascending=False).reset_index(drop=True)
total = df_network.IMPORTE_PESOS.sum()
df_network = df_network.assign(weight=(df_network.IMPORTE_PESOS * 100).divide(total))
df_network = df_network.assign(weight_cumsum=df_network.weight.cumsum())
df_network = df_network.loc[df_network.weight_cumsum <= 90]
df_network = df_network.drop(['weight', 'weight_cumsum'], axis=1)
print(df_network.shape, df_network.PROVEEDOR_CONTRATISTA.nunique())
df_network.head()

(2577, 3) 1579


,CLAVEUC,PROVEEDOR_CONTRATISTA,IMPORTE_PESOS
0,018TOQ054,AVANZIA INSTALACIONES SA DE CV,1.269174e+10
1,018TOQ755,ANGEL CONSTRUCTORA INTEGRAL Y DE SERVICIOS SA ...,9.400878e+09
2,018TOQ003,SIEMENS INNOVACIONES SA DE CV,9.282063e+09
3,018TOQ054,ISOLUX DE MEXICO SA DE CV,8.805876e+09
4,018TOQ054,SENERMEX INGENIERIA Y SISTEMAS SA DE CV,7.323606e+09


In [19]:
def get_unique_pair(df, column_x: str, column_y: str):
    data = []
    for x, y in zip(df[column_x], df[column_y]):
        if y > x:
            data.append(x + '|' + y)
        else:
            data.append(y + '|' + x)
    return df.assign(pair_id=pd.Series(data, dtype=str, index=df.index))

def split_pair(df, col: str):
    PoC_x, PoC_y = zip(*df[col].str.split('|').values)
    PoC_x = pd.Series(PoC_x, index=df.index)
    PoC_y = pd.Series(PoC_y, index=df.index)
    return df.assign(PoC_x=PoC_x, PoC_y=PoC_y)
    

df_network = pd.merge(df_network, df_network, on='CLAVEUC', how='inner')
# df_network = df_network.drop('CLAVEUC', axis=1)
df_network = df_network.loc[df_network.PROVEEDOR_CONTRATISTA_x != df_network.PROVEEDOR_CONTRATISTA_y, :]

# falta lo del pair id
df_network = get_unique_pair(df_network, 'PROVEEDOR_CONTRATISTA_x', 'PROVEEDOR_CONTRATISTA_y')
df_network = df_network.drop_duplicates(subset=['pair_id', 'CLAVEUC'])
df_network = df_network.assign(weight=(df_network.IMPORTE_PESOS_x * df_network.IMPORTE_PESOS_y))

df_network = df_network.groupby('pair_id').weight.sum()
df_network = df_network.sort_values(ascending=False)
df_network = df_network.reset_index()
df_network = split_pair(df_network, 'pair_id')
df_network = df_network.drop('pair_id', axis=1)
df_network.loc[:, 'weight'] = df_network.weight.divide(df_network.weight.min())
print(df_network.shape)
df_network.head()

(42111, 3)


,weight,PoC_x,PoC_y
0,1.128763e+06,AVANZIA INSTALACIONES SA DE CV,ISOLUX DE MEXICO SA DE CV
1,9.387609e+05,AVANZIA INSTALACIONES SA DE CV,SENERMEX INGENIERIA Y SISTEMAS SA DE CV
2,9.364959e+05,AVANZIA INSTALACIONES SA DE CV,ELECNOR SA
3,7.342194e+05,AVANZIA INSTALACIONES SA DE CV,OMEGA CONSTRUCCIONES INDUSTRIALES SA DE CV
4,7.043243e+05,AVANZIA INSTALACIONES SA DE CV,EDEMTEC SA DE CV


In [8]:
df_network.shape, df_network.PoC_x.nunique()

((182123, 3), 3373)

In [8]:
# IUSA
# CONTROLES Y MEDIDORES ESPECIALIZADOS SA DE CV
# df_network.loc[df_network.PROVEEDOR_CONTRATISTA_x.str.startswith('IUSA')]

In [20]:
G = nx.from_pandas_dataframe(df_network, 'PoC_x', 'PoC_y', 'weight', nx.Graph())
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1545
Number of edges: 42111
Average degree:  54.5126


In [21]:
# 
grados = nx.degree_centrality(G)
grados = pd.Series(grados).sort_values(ascending=False)
grados.name = 'simple_degree'
scaler = MinMaxScaler((20, 150))
new_grados = scaler.fit_transform(grados.values.reshape(-1, 1))
grados = {n: float(val) for n, val, in zip(grados.index, new_grados.flatten())}

weight_grados =  G.degree(weight='weight')
weight_grados = pd.Series(weight_grados).sort_values(ascending=False)
weight_grados.name = 'weighted_degree'
scaler = MinMaxScaler((20, 150))
new_weight_grados = scaler.fit_transform(weight_grados.values.reshape(-1, 1))
weight_grados = {n: float(val) for n, val, in zip(weight_grados.index, new_weight_grados.flatten())}

# montos
montos_att = df_empresas.set_index('PROVEEDOR_CONTRATISTA').loc[(G.nodes()), 'MONTO_PESOS'].to_dict()
montos_att = {n: float(v) for n, v in montos_att.items()}

# montos size
montos_size = pd.Series(montos_att)
montos_size = montos_size.divide(montos_size.min())
montos_size = montos_size.sort_values(ascending=False)
scaler = MinMaxScaler((20, 150))
new_size = scaler.fit_transform(montos_size.values.reshape(-1, 1))
montos_size = {n: float(v) for n, v in zip(montos_size.index, new_size.flatten())}

## Agregar los otros degrees y el monto

In [22]:
nx.set_node_attributes(G, 'simple_degree', grados)
nx.set_node_attributes(G, 'weighted_degree', weight_grados)
nx.set_node_attributes(G, 'montos', montos_att)
nx.set_node_attributes(G, 'montos_size', montos_size)

In [23]:
nx.write_graphml(G, '../data/processed/CFE_PoCs.graphml')

In [15]:
pd.Series(montos_att).sort_values(ascending=False)

SIXSIGMA NETWORKS MEXICO                                                       1.988804e+10
SEGUROS INBURSA SA GRUPO FINANCIERO INBUR SA                                   1.788541e+10
SIEMENS INNOVACIONES SA DE CV                                                  1.486462e+10
AVANZIA INSTALACIONES SA DE CV                                                 1.269174e+10
METLIFE MEXICO, S.A.                                                           1.141287e+10
CASANOVA VALLEJO SA DE CV                                                      9.771139e+09
ISOLUX DE MEXICO SA DE CV                                                      9.440375e+09
ANGEL CONSTRUCTORA INTEGRAL Y DE SERVICIOS SA DE CV                            9.408329e+09
EDENRED MEXICO SA DE CV                                                        9.278551e+09
SI VALE MEXICO SA DE CV                                                        9.271543e+09
SENERMEX INGENIERIA Y SISTEMAS SA DE CV                                        8

In [29]:
# nodos = set(df_network.PoC_x.unique()) | set(df_network.PoC_y.unique())

In [48]:
((np.log(montos_size) + 5) * 10).describe()

count    3490.000000
mean       70.102211
std        15.462968
min        50.000000
25%        58.507157
50%        66.675398
75%        77.507931
max       138.624995
dtype: float64

In [24]:
grados.describe()

count    3.490000e+03
mean     4.166787e+18
std      4.036380e+19
min      1.153108e+13
25%      3.666433e+15
50%      1.724627e+16
75%      7.634375e+16
max      1.035331e+21
Name: simple_degree, dtype: float64

In [34]:
df_network.loc[df_network.pair_id == 'OROPEZA INGENIEROS SA DE CV|SCHWEITZER ENGINEERING LABORATORIES SA DE CV'].CLAVEUC.unique().shape

(37,)

In [23]:
df_network.shape

(2989346, 6)

In [24]:
df_network.drop_duplicates(subset=['pair_id']).shape, df_network.drop_duplicates(subset=['pair_id', 'CLAVEUC']).shape

((1271388, 6), (1494673, 6))

In [9]:
# df_network.groupby(['pair_id']).PROVEEDOR_CONTRATISTA_x.count().sort_values(ascending=False)

In [9]:
df_network.dtypes

PROVEEDOR_CONTRATISTA_x     object
IMPORTE_PESOS_x            float64
PROVEEDOR_CONTRATISTA_y     object
IMPORTE_PESOS_y            float64
dtype: object

In [ ]:
df

## Network por dependencia (top 5)

In [5]:
dfs_deps = {}

for dep in top_5_dep:
    
    df_network = df_validos.loc[
        df_validos.DEPENDENCIA == dep, ['CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'IMPORTE_PESOS']
    ].groupby(
        ['CLAVEUC', 'PROVEEDOR_CONTRATISTA']
    ).IMPORTE_PESOS.sum().reset_index().sort_values('IMPORTE_PESOS', ascending=False).reset_index(drop=True)
    total = df_network.IMPORTE_PESOS.sum()
    df_network = df_network.assign(weight=(df_network.IMPORTE_PESOS * 100).divide(total))
    df_network = df_network.loc[df_network.weight.cumsum() < 99]
    print('Num nodos:', df_network.CLAVEUC.unique().shape[0] + df_network.PROVEEDOR_CONTRATISTA.unique().shape[0])
    dfs_deps[dep] = df_network

Num nodos: 3431
Num nodos: 10660
Num nodos: 6144
Num nodos: 1498
Num nodos: 5495


In [6]:
redes = {}
for dep, df_network in dfs_deps.items():
    unidades = {u: 'Unidad' for u in df_network.CLAVEUC.unique()}
    pocs = {p: 'PoC' for p in df_network.PROVEEDOR_CONTRATISTA.unique()}
    
    tipo_att = {**unidades, **pocs}
    
    # Size de DEPENDENCIAS y de POC
    monto_pocs = df_validos.loc[
        (df_validos.PROVEEDOR_CONTRATISTA.isin(df_network.PROVEEDOR_CONTRATISTA.unique()))
    ].groupby('PROVEEDOR_CONTRATISTA').IMPORTE_PESOS.sum()

    monto_uni = df_validos.loc[
        (df_validos.CLAVEUC.isin(df_network.CLAVEUC.unique()))
    ].groupby('CLAVEUC').IMPORTE_PESOS.sum()

    minimo = min(monto_uni.min(), monto_pocs.min())

    monto_pocs = monto_pocs.divide(minimo)
    monto_uni = monto_uni.divide(minimo)

    monto_pocs = {k: float(v) for k, v in monto_pocs.to_dict().items()}
    monto_uni = {k: float(v) for k, v in monto_uni.to_dict().items()}

    size_att = {**monto_uni, **monto_pocs}

    df_size = pd.Series(size_att)
    df_size.name = 'monto'
    df_size.index.name = 'Nodo'
    df_size = (df_size.apply(np.log) + 1) * 6

    size_att = df_size.sort_values(ascending=False).to_dict()
    size_att = {k: float(v) for k, v in size_att.items()}

    forma_att = {k: 'Circulo' for k in size_att}
    print('Size att:', len(size_att))
    G = nx.from_pandas_dataframe(df_network, 'CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'weight', nx.DiGraph())
    nx.set_node_attributes(G, 'Tipo', tipo_att)
    nx.set_node_attributes(G, 'Forma', forma_att)
    nx.set_node_attributes(G, 'Montos', size_att)
    print(nx.info(G))
    
    # Falta agregar el nombre de la unidad y monto para que se vea en cytoscope
    redes[dep] = G

Size att: 3431
Name: 
Type: DiGraph
Number of nodes: 3431
Number of edges: 4140
Average in degree:   1.2066
Average out degree:   1.2066
Size att: 10660
Name: 
Type: DiGraph
Number of nodes: 10660
Number of edges: 21328
Average in degree:   2.0008
Average out degree:   2.0008
Size att: 6144
Name: 
Type: DiGraph
Number of nodes: 6144
Number of edges: 10825
Average in degree:   1.7619
Average out degree:   1.7619
Size att: 1498
Name: 
Type: DiGraph
Number of nodes: 1498
Number of edges: 2085
Average in degree:   1.3919
Average out degree:   1.3919
Size att: 5495
Name: 
Type: DiGraph
Number of nodes: 5495
Number of edges: 7210
Average in degree:   1.3121
Average out degree:   1.3121


In [ ]:
df_cfe

In [11]:
df_cfe = dfs_deps['COMISION FEDERAL DE ELECTRICIDAD']

In [14]:
df_cfe[df_cfe.PROVEEDOR_CONTRATISTA.str.startswith('IUSA MEDICION')]

,CLAVEUC,PROVEEDOR_CONTRATISTA,IMPORTE_PESOS,weight
78,018TOQ003,IUSA MEDICION,8.293185e+08,0.211520
292,018TOQ008,IUSA MEDICION,1.336272e+08,0.034082
362,018TOQ988,IUSA MEDICION,1.072654e+08,0.027358
377,018TOQ990,IUSA MEDICION,9.922108e+07,0.025307
404,018TOQ062,IUSA MEDICION,9.035283e+07,0.023045
406,018TOQ066,IUSA MEDICION,9.017063e+07,0.022998
409,018TOQ015,IUSA MEDICION,8.965123e+07,0.022866
430,018TOQ997,IUSA MEDICION,8.552908e+07,0.021814
435,018TOQ011,IUSA MEDICION,8.491362e+07,0.021657
471,018TOQ004,IUSA MEDICION,7.816292e+07,0.019936


In [10]:
dfs_deps['COMISION FEDERAL DE ELECTRICIDAD'].[]

,CLAVEUC,PROVEEDOR_CONTRATISTA,IMPORTE_PESOS,weight
0,018TOQ054,AVANZIA INSTALACIONES SA DE CV,1.269174e+10,3.237062
1,018TOQ755,ANGEL CONSTRUCTORA INTEGRAL Y DE SERVICIOS SA ...,9.400878e+09,2.397719
2,018TOQ003,SIEMENS INNOVACIONES SA DE CV,9.282063e+09,2.367415
3,018TOQ054,ISOLUX DE MEXICO SA DE CV,8.805876e+09,2.245962
4,018TOQ054,SENERMEX INGENIERIA Y SISTEMAS SA DE CV,7.323606e+09,1.867905
5,018TOQ054,ELECNOR SA,7.305936e+09,1.863398
6,018TOQ003,TRAFIGURA BEHEER B.V. AMSTERDAM BRANCH OFFICE ...,6.458986e+09,1.647381
7,018TOQ003,GLENCORE INTERNATIONAL AG,5.921092e+09,1.510190
8,018TOQ003,AGB MATERIALS SA DE CV,5.909603e+09,1.507260
9,018TOQ054,OMEGA CONSTRUCCIONES INDUSTRIALES SA DE CV,5.727905e+09,1.460917


In [9]:
nx.write_graphml(redes['COMISION FEDERAL DE ELECTRICIDAD'], '../data/processed/CFE_UC_PoCs.graphml')

In [7]:
redes.keys()

dict_keys(['INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE LOS TRABAJADORES DEL ESTADO', 'COMISION FEDERAL DE ELECTRICIDAD', 'INSTITUTO MEXICANO DEL SEGURO SOCIAL', 'PETROLEOS MEXICANOS (CORPORATIVO)', 'SECRETARIA DE COMUNICACIONES Y TRANSPORTES'])

## En procedimientos hay 1390 con claveuc, no sabemos qué unidad lo hizo

## Hay que hacer algo con los contratos que reportan 0 pesos y revisar que no es mi culpa

In [4]:
df_validos.loc[df_validos.CLAVEUC.str.len() != 9, :].shape

(1390, 43)

In [26]:
df_validos.loc[df_validos.CLAVEUC.str.len() != 9, :].IMPORTE_PESOS.sum()

2987465340.7129803

In [24]:
df_validos.loc[df_validos.CLAVEUC.str.len() != 9, :].DEPENDENCIA.unique()

array(['SERVICIO DE ADMINISTRACION TRIBUTARIA',
       'PROCURADURIA FEDERAL DE PROTECCION AL AMBIENTE'], dtype=object)

In [13]:
# df_validos.CLAVEUC.str.len().value_counts()

In [18]:
# df_validos.loc[df_validos.CLAVEUC.str.len() != 9, :]

In [25]:
test = df_validos[df_validos.CLAVEUC.str.len() == 3]
test

,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,IMPORTE_PESOS
173569,SERVICIO DE ADMINISTRACION TRIBUTARIA,SAT,SAT-ADMINISTRACION DE OPERACION DE RECURSOS Y ...,600,FERNANDO MENDOZA ELVIRA,MICRO,nan,CORPORACION DE PROFESIONALES EN ZOOTECNIA SA D...,HABILITADO,MICRO,NaN,MXN,SA-006E00002-N5-2014,None,ADJUDICACION DIRECTA FEDERAL,456726,PRODUCTOS ALIMENTICIOS PARA ANIMALES,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,NaN,NaT,NaT,NaT,NaT,2013-01-02,2013-12-31,533858,"CORPORACION DE PROFESIONALES EN ZOOTECNIA, S. ...",12. Adjudicación Directa Nacional Simplificada...,None,PoC,None,Si,None,NaN,None,NaN,209340.00
173570,SERVICIO DE ADMINISTRACION TRIBUTARIA,SAT,SAT-ADMINISTRACION DE OPERACION DE RECURSOS Y ...,600,FERNANDO MENDOZA ELVIRA,MEDIANA,nan,GENERAL SPORTS SA DE CV,HABILITADO,MEDIANA,NaN,MXN,SA-006E00002-N204-2013,None,ADJUDICACION DIRECTA FEDERAL,463701,ARRENDAMIENTO DE MOBILIARIO,None,SERVICIOS,EXPIRADO,0.0,0.0,Nacional,0.0,NaN,NaT,NaT,NaT,NaT,2013-09-21,2013-09-21,539147,ARRENDAMIENTO DE MOBILIARIO,12. Adjudicación Directa Nacional Simplificada...,None,PoC,None,Si,None,NaN,None,NaN,46450.00
173571,SERVICIO DE ADMINISTRACION TRIBUTARIA,SAT,SAT-ADMINISTRACION DE OPERACION DE RECURSOS Y ...,600,FERNANDO MENDOZA ELVIRA,PEQUENA,nan,CIRCULO Y MEDIO SA,HABILITADO,PEQUENA,NaN,MXN,SA-006E00002-N202-2013,None,ADJUDICACION DIRECTA FEDERAL,462949,SERVICIOS PARA CAPACITACION A SERVIDORES PUBLICOS,None,SERVICIOS,EXPIRADO,0.0,0.0,Nacional,0.0,NaN,NaT,NaT,NaT,NaT,2013-12-02,2013-12-31,538574,SERVICIO PARA CAPACITACION A SERVIDORES PUBLICOS,12. Adjudicación Directa Nacional Simplificada...,None,PoC,None,Si,None,NaN,None,NaN,790399.95
173572,SERVICIO DE ADMINISTRACION TRIBUTARIA,SAT,SAT-ADMINISTRACION DE OPERACION DE RECURSOS Y ...,600,FERNANDO MENDOZA ELVIRA,MICRO,nan,MANUFACTURAS METALICAS VITALI SA DE CV,HABILITADO,MICRO,NaN,MXN,SA-006E00002-N203-2013,None,ADJUDICACION DIRECTA FEDERAL,463321,ARTICULOS DEPORTIVOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,NaN,NaT,NaT,NaT,NaT,2013-09-12,2013-09-19,538882,ARTICULOS DEPORTIVOS,12. Adjudicación Directa Nacional Simplificada...,None,PoC,None,Si,None,NaN,None,NaN,180000.00
173573,SERVICIO DE ADMINISTRACION TRIBUTARIA,SAT,SAT-ADMINISTRACION DE OPERACION DE RECURSOS Y ...,600,FERNANDO MENDOZA ELVIRA,PEQUENA,nan,COMERCIALIZADORA ENTUSMARCAS SA DE CV,HABILITADO,PEQUENA,NaN,MXN,SA-006E00002-N220-2013,None,ADJUDICACION DIRECTA FEDERAL,467080,ARTICULOS DEPORTIVOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,NaN,NaT,NaT,NaT,NaT,2013-09-16,2013-09-19,541900,ARTICULOS DEPORTIVOS,12. Adjudicación Directa Nacional Simplificada...,None,PoC,None,Si,None,NaN,None,NaN,188000.00
173574,SERVICIO DE ADMINISTRACION TRIBUTARIA,SAT,SAT-ADMINISTRACION DE OPERACION DE RECURSOS Y ...,600,FERNANDO MENDOZA ELVIRA,MEDIANA,nan,GIMSA CONSTRUCCIONES INTEGRALES DEL GOLFO SA D...,HABILITADO,MEDIANA,NaN,MXN,SA-006E00002-N219-2013,None,ADJUDICACION DIRECTA FEDERAL,467032,SERVICIO DE MANTENIMIENTO Y CONSERVACION .,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,NaN,NaT,NaT,NaT,NaT,2013-08-21,2014-01-26,541859,MANTENIMIENTO Y CONSERVACION DE INMUEBLES,12. Adjudicación Directa Nacional Simplificada...,None,PoC,None,Si,None,NaN,None,NaN,21512499.31
173575,SERVICIO DE ADMINISTRACION TRIBUTARIA,SAT,SAT-ADMINISTRACION DE OPERACION DE RECURSOS Y ...,600,FERNANDO MENDOZA ELVIRA,MICRO,nan,MARIA GUADALUPE SOLIS DIAZ,HABILITADO,NO MIPYME,NaN

In [12]:
# test[test.SIGLAS == 'SAT'].NOMBRE_DE_LA_UC.values

In [ ]:
df_validos

In [5]:
## Leer el registro único
# clave UC es de 9 caracteres
df_dir_fed = pd.read_excel('../data/raw/Directorio Unidades Compradoras Gobierno Federal.xls', dtype=str)
print(df_dir_fed.shape)
df_dir_fed.head()

(1930, 12)


,Nombre UC en CompraNet,Clave UC en CompraNet,Pertenece al,Dependencia o Entidad,Siglas,Página Web,Dirección,CP,Entidad Federativa,Delegación o municipio,Telefono de la UC,Nombre del Responsable
0,AFSEDF-ADMINISTRACIÓN FEDERAL DE SERVICIOS EDU...,011C00999,APF,Administración Federal de Servicios Educativos...,AFSEDF,www.sep.gob.mx,Río Rhin 9 Piso 8,06500,Ciudad de México,Cuauhtémoc,36017100 ext. 40302,Ricardo Rafael Nieto Hernández
1,API-Altamira-Departamento de Concursos y Contr...,009J2Y001,APF,"Administración Portuaria Integral de Altamira,...",API-Altamira,www.puertoaltamira.com.mx,"Calle Rios Tamesi Km 0 800, lado sur, Colonia ...",89603,Tamaulipas,Altamira,18332606060,Alberto Trejo Saldaña
2,API-Altamira-DEPARTAMENTO DE RECURSOS MATERIAL...,009J2Y002,APF,"Administración Portuaria Integral de Altamira,...",API-Altamira,www.puertoaltamira.com.mx,RIO TAMESI KM 0 800 LADO SUR COLONIA PUERTO IN...,89603,Tamaulipas,Altamira,8332606060,Denisse Ramirez Diaz
3,API-Coatzacoalcos-Coatzacoalcos Jefatura de Pr...,009J3F001,APF,Administración Portuaria Integral de Coatzacoa...,API-Coatzacoalcos,http://www.puertocoatzacoalcos.com.mx/,Interior del Recinto Fiscal s/n antes zona franca,96400,Veracruz,Coatzacoalcos,019212110270,José Manuel Kin Galindo
4,API-Coatzacoalcos-Administración Portuaria Int...,009J3F002,APF,Administración Portuaria Integral de Coatzacoa...,API-Coatzacoalcos,www.puertocoatzacoalcos.com.mx,Interior del Recinto Fiscal Zona Franca S/N,96400,Veracruz,Coatzacoalcos,9212110270,Bernardino Guzmán Zavala


In [11]:
# df_dir_fed[df_dir_fed.Siglas == 'SAT']

In [ ]:
from io import StringIO
df_test = """L1,L2,w
1,A,0.9
2,A,0.2
3,A,0.5
1,B,0.9
2,B,0.8"""
df_test = pd.read_csv(StringIO(df_test))
print(df_test)
df2 = pd.merge(df_test, df_test, on='L2')
print(df2)
df3 = df2.loc[ df2.L1_x > df2.L1_y ]
df3